In [1]:
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
%load_ext autoreload
%autoreload 1
from imports import *
from functions import *

Get Available Stations ID List

In [3]:
# get all stations and some metadata as a Pandas DataFrame
stations_df = api.stations()
# parse the response as a dictionary
stations_df = api.stations(as_df=True)

print(len(stations_df))

146


Filter Buoys by Remarks

In [4]:
access_error_url_list = []

# Liste de mots à rechercher dans la colonne "Remark"
blacklist = ["Failure", "ceased", "failed", "recovered", "stopped", 'adrift']
stations_id_set = set()

print(f'Avant Filtre: {stations_df.shape[0]}')

# Liste pour collecter les indices à supprimer
indices_a_supprimer = []

# Parcours des lignes de la DataFrame
for idx, row in stations_df.iterrows():
    station_id = row["Station"]
    station_Location = row["Hull No./Config and Location"]  # Extraire la valeur de la cellule pour chaque ligne
    
    # Extraction du nom de la station si un ")" est trouvé
    if ")" in station_Location:
        station_name = station_Location.split(')')[1].rstrip(" )")  # On enlève l'espace et la parenthèse en fin de chaîne
    else:
        station_name = station_Location.strip()  # Si pas de ")", on garde toute la chaîne

    station_name = station_name.rstrip(" )").replace("(", "").replace(")", "").strip()

    # Nettoyage final pour enlever toute parenthèse ou espace en fin de nom
    station_name = station_name.rstrip(" )")

    # Vérifier si "Remark" n'est pas NaN et si un des éléments de blacklist est dans "Remark"
    if isinstance(row["Remark"], str) and any(blacklist_word.lower() in row["Remark"].lower() for blacklist_word in blacklist):
        # Ajouter l'index à la liste
        indices_a_supprimer.append(idx)
        url = get_buoy_url(station_id)
        access_error_url_list.append(url)
    else:
        pass
# Supprimer les lignes après la boucle
stations_df.drop(index=indices_a_supprimer, inplace=True)

print(f'Après Filtre: {stations_df.shape[0]}')

Avant Filtre: 146
Après Filtre: 43


Build Buoys_datas Dict

In [5]:
# Dictionnaire pour stocker les DataFrames, clé : ID de la bouée, valeur : DataFrame
buoy_datas = {}
buoy_list = []

# Parcours de chaque bouée dans stations_df
for index, row in stations_df.iterrows():
    buoy_id = row['Station']
    metadata = get_station_metadata(buoy_id)

    # ✅ Récupérer les données sous forme de dictionnaire
    buoy_info = parse_buoy_json(metadata)

    # ✅ Stocker directement les données dans buoy_datas
    buoy_datas[buoy_id] = buoy_info
    buoy_list.append(buoy_id)

# Affichage du nombre de bouées réussies et échouées
print(f"Nombre de bouées traitées : {len(buoy_datas)}\n")

# Afficher le contenu de buoy_datas

first_key =next(iter(buoy_datas))
first_key
buoy_datas[first_key]


🔍 Début du parsing de la bouée...
🌍 Zone de la station : grays reef
🆔 Station ID : 41008
✅ Coordonnées extraites : Latitude = 31.40N, Longitude = 80.87W
🌊 Water Depth : 16 m
🌡️ Sea Temp Depth : 2
🌬️ Barometer Elevation (m): 2.4
💨 Anemometer Height : 3.8
🌤️ Air Temp Height : 3.4
🔗 URL de la bouée : https://www.ndbc.noaa.gov/station_page.php?station=41008
✅ Parsing terminé !


🔍 Début du parsing de la bouée...
🌍 Zone de la station : ne st martin
🆔 Station ID : 41044
✅ Coordonnées extraites : Latitude = 21.58N, Longitude = 58.63W
🌊 Water Depth : 5419 m
🌡️ Sea Temp Depth : 2
🌬️ Barometer Elevation (m): 2.4
💨 Anemometer Height : 3.8
🌤️ Air Temp Height : 3.4
🔗 URL de la bouée : https://www.ndbc.noaa.gov/station_page.php?station=41044
✅ Parsing terminé !


🔍 Début du parsing de la bouée...
🌍 Zone de la station : south bermuda
🆔 Station ID : 41049
✅ Coordonnées extraites : Latitude = 27.50N, Longitude = 62.27W
🌊 Water Depth : 5480 m
🌡️ Sea Temp Depth : 1.5
🌬️ Barometer Elevation (m): 2.7
💨 An

{'station_zone': 'grays reef',
 'lat_buoy': '31.40N',
 'lon_buoy': '80.87W',
 'Water_depth': '16 m',
 'sea_temp_depth': '2',
 'Barometer_elevation': '2.4',
 'Anemometer_height': '3.8',
 'Air_temp_height': '3.4',
 'url': 'https://www.ndbc.noaa.gov/station_page.php?station=41008'}

Collecte de données marines et météos

In [6]:
# 🚀 Démarrage du processus
print("\n🚀 Démarrage du processus de collecte des données...\n")

# Initialisation des compteurs
marine_data_collected_successfully = marine_data_collected_failed = 0
meteo_data_collected_successfully = meteo_data_collected_failed = 0

success = False
total_stations = stations_df.shape[0]
count = 0

# 🔄 Parcours des bouées / stations
for idx, row in stations_df.iterrows():
    buoy_id = row["Station"]

    ######### 🌊 MARINE DATA #########
    try:
        df_marine = NDBC.realtime_observations(buoy_id)
        if df_marine is None or df_marine.empty:
            marine_data_collected_failed += 1
            continue

        marine_data_collected_successfully += 1
    except Exception as e:
        print(f"⚠️ Erreur collecte marine {buoy_id}: {e}")
        marine_data_collected_failed += 1
        continue

    # Ajout des métadonnées
    try:
        buoy_info = buoy_datas.get(buoy_id, {})
        Lat, Lon = buoy_info.get('lat_buoy'), buoy_info.get('lon_buoy')
        if Lat is None or Lon is None:
            raise ValueError(f"Données manquantes pour {buoy_id}")

        df_marine['Lat'] = Lat
        df_marine['Lon'] = Lon
        df_marine['Water_depth'] = buoy_info.get('Water_depth', None)
        df_marine.columns = ['Datetime' if 'date' in col.lower() or 'time' in col.lower() else col for col in df_marine.columns]
        df_marine['Datetime'] = df_marine['Datetime'].dt.tz_localize(None)

        buoy_datas[buoy_id]["Marine"] = df_marine

        station_zone = safe_get(parse_buoy_json(get_station_metadata(buoy_id)), "station_zone")
        Bronze_Marine_table_Name = f"br_{buoy_id}_marine_{station_zone}".replace('.', '_').replace('-', '_').replace(' ', '_').lower()

    except Exception as e:
        print(f"⚠️ Erreur métadonnées marine {buoy_id}: {e}")
        marine_data_collected_failed += 1
        continue

    ######### ⛅ METEO DATA #########
    try:
        df_meteo = meteo_api_request([Lat, Lon])
        if df_meteo is None or df_meteo.empty:
            meteo_data_collected_failed += 1
            continue
        
        rename_columns(df_meteo, {'date':'Datetime'})
        df_meteo.columns = ['Datetime' if 'date' in col.lower() or 'time' in col.lower() else col for col in df_meteo.columns]
        df_meteo['Datetime'] = df_meteo['Datetime'].dt.tz_localize(None)
    
        buoy_datas[buoy_id]["Meteo"] = df_meteo
        meteo_data_collected_successfully += 1
    except Exception as e:
        print(f"⚠️ Erreur collecte météo {buoy_id}: {e}")
        meteo_data_collected_failed += 1
        continue

# Retirer les bouées avec des DataFrames vides ou None
buoy_datas = {buoy_id: data for buoy_id, data in buoy_datas.items() 
              if "Marine" in data and data["Marine"] is not None and not data["Marine"].empty
              and "Meteo" in data and data["Meteo"] is not None and not data["Meteo"].empty}

# 🔚 Résumé final

print("\n📝 Résumé final :")
print(f"🌊 Marine - Collecte ✅ {marine_data_collected_successfully} ❌ {marine_data_collected_failed}")
print(f"⛅ Météo - Collecte ✅ {meteo_data_collected_successfully} ❌ {meteo_data_collected_failed}")

# Afficher la longueur du dictionnaire (nombre de bouées avec des données valides)
print(f"\n📊 Nombre de bouées avec des données valides : {len(buoy_datas)}")


🚀 Démarrage du processus de collecte des données...


🔍 Début du parsing de la bouée...
🌍 Zone de la station : grays reef
🆔 Station ID : 41008
✅ Coordonnées extraites : Latitude = 31.40N, Longitude = 80.87W
🌊 Water Depth : 16 m
🌡️ Sea Temp Depth : 2
🌬️ Barometer Elevation (m): 2.4
💨 Anemometer Height : 3.8
🌤️ Air Temp Height : 3.4
🔗 URL de la bouée : https://www.ndbc.noaa.gov/station_page.php?station=41008
✅ Parsing terminé !

📊 station_zone : grays reef
🔄 Colonne 'date' renommée en 'Datetime'
✅ Colonnes renommées : {'date': 'Datetime'}

🔍 Début du parsing de la bouée...
🌍 Zone de la station : ne st martin
🆔 Station ID : 41044
✅ Coordonnées extraites : Latitude = 21.58N, Longitude = 58.63W
🌊 Water Depth : 5419 m
🌡️ Sea Temp Depth : 2
🌬️ Barometer Elevation (m): 2.4
💨 Anemometer Height : 3.8
🌤️ Air Temp Height : 3.4
🔗 URL de la bouée : https://www.ndbc.noaa.gov/station_page.php?station=41044
✅ Parsing terminé !

📊 station_zone : ne st martin
🔄 Colonne 'date' renommée en 'Datetime'
✅ Co

In [7]:
display_buoys_missing_df_counts(buoy_datas)


🌊 Nombre de bouées sans données 'Marine' : 0/41

☁️ Nombre de bouées sans données 'Meteo' : 0/41


Data Enrichment with MetaDatas

In [8]:
list_not_include = ['lon_buoy', "lat_buoy", "url"]
for buoy_id, value in buoy_datas.items():
    print(f"\n🔍 Traitement de la Station ID: {buoy_id}")

    marine_df = buoy_datas[buoy_id]["Marine"]
    meteo_df = buoy_datas[buoy_id]["Meteo"]

    try:
        # Récupérer les métadonnées de la station
        buoy_metadata = get_station_metadata(buoy_id)
        parsed_data = parse_buoy_json(buoy_metadata)

        # Mise à jour du dictionnaire avec les métadonnées
        data = buoy_datas[buoy_id]
        data.update(parsed_data)
        
        # Ajouter les métadonnées comme nouvelles colonnes dans marine_df
        if marine_df is not None:
            marine_df["Station ID"] = str(buoy_id)
            for key, value in parsed_data.items():
                # Vérifier si la clé n'est pas dans la liste des exclusions
                if key not in list_not_include:
                    marine_df[key] = value
                    print(f"✅ Colonne '{key}' ajoutée au DataFrame de la station {buoy_id}")

    except Exception as e:
        print(f"❌ Erreur pour la station {buoy_id}: {e}")

# Vérification de l'ajout des colonnes en prenant un id au hasard
station_id = random.choice(list(buoy_datas.keys()))
marine_df = buoy_datas[station_id]["Marine"]

if marine_df is not None:
    print("\nColonnes ajoutées au DataFrame de la station", station_id)
    print(marine_df.columns)


🔍 Traitement de la Station ID: 41008

🔍 Début du parsing de la bouée...
🌍 Zone de la station : grays reef
🆔 Station ID : 41008
✅ Coordonnées extraites : Latitude = 31.40N, Longitude = 80.87W
🌊 Water Depth : 16 m
🌡️ Sea Temp Depth : 2
🌬️ Barometer Elevation (m): 2.4
💨 Anemometer Height : 3.8
🌤️ Air Temp Height : 3.4
🔗 URL de la bouée : https://www.ndbc.noaa.gov/station_page.php?station=41008
✅ Parsing terminé !

✅ Colonne 'station_zone' ajoutée au DataFrame de la station 41008
✅ Colonne 'Water_depth' ajoutée au DataFrame de la station 41008
✅ Colonne 'sea_temp_depth' ajoutée au DataFrame de la station 41008
✅ Colonne 'Barometer_elevation' ajoutée au DataFrame de la station 41008
✅ Colonne 'Anemometer_height' ajoutée au DataFrame de la station 41008
✅ Colonne 'Air_temp_height' ajoutée au DataFrame de la station 41008

🔍 Traitement de la Station ID: 41044

🔍 Début du parsing de la bouée...
🌍 Zone de la station : ne st martin
🆔 Station ID : 41044
✅ Coordonnées extraites : Latitude = 21.58

In [9]:
df_marine.head()

,wind_direction,wind_speed,wind_gust,wave_height,dominant_wave_period,average_wave_period,dominant_wave_direction,pressure,air_temperature,water_temperature,...,Datetime,Lat,Lon,Water_depth,Station ID,station_zone,sea_temp_depth,Barometer_elevation,Anemometer_height,Air_temp_height
0,350.0,6.7,7.7,NaN,NaN,NaN,NaN,1021.4,2.3,NaN,...,2025-04-11 12:00:00,41.63N,82.84W,N/A,SBIO1,"south bass island, oh",None,178.4,24.3,23.4
1,360.0,4.1,6.7,NaN,NaN,NaN,NaN,1020.9,2.5,NaN,...,2025-04-11 11:00:00,41.63N,82.84W,N/A,SBIO1,"south bass island, oh",None,178.4,24.3,23.4
2,350.0,5.1,6.7,NaN,NaN,NaN,NaN,1020.4,2.9,NaN,...,2025-04-11 10:00:00,41.63N,82.84W,N/A,SBIO1,"south bass island, oh",None,178.4,24.3,23.4
3,360.0,2.6,4.1,NaN,NaN,NaN,NaN,1020.3,2.8,NaN,...,2025-04-11 09:00:00,41.63N,82.84W,N/A,SBIO1,"south bass island, oh",None,178.4,24.3,23.4
4,360.0,4.1,6.7,NaN,NaN,NaN,NaN,1019.6,2.6,NaN,...,2025-04-11 08:00:00,41.63N,82.84W,N/A,SBIO1,"south bass island, oh",None,178.4,24.3,23.4


In [10]:
display_buoys_missing_df_counts(buoy_datas)


🌊 Nombre de bouées sans données 'Marine' : 0/41

☁️ Nombre de bouées sans données 'Meteo' : 0/41


In [11]:
display(df_marine.columns)
display(df_meteo.columns)

Index(['wind_direction', 'wind_speed', 'wind_gust', 'wave_height',
       'dominant_wave_period', 'average_wave_period',
       'dominant_wave_direction', 'pressure', 'air_temperature',
       'water_temperature', 'dewpoint', 'visibility', '3hr_pressure_tendency',
       'water_level_above_mean', 'Datetime', 'Lat', 'Lon', 'Water_depth',
       'Station ID', 'station_zone', 'sea_temp_depth', 'Barometer_elevation',
       'Anemometer_height', 'Air_temp_height'],
      dtype='object')

Index(['Datetime', 'temperature_2m', 'relative_humidity_2m', 'dew_point_2m',
       'precipitation', 'rain', 'showers', 'pressure_msl', 'surface_pressure',
       'cloud_cover', 'cloud_cover_low', 'cloud_cover_mid', 'cloud_cover_high',
       'visibility', 'wind_speed_10m', 'soil_temperature_0cm',
       'soil_moisture_0_to_1cm', 'is_day'],
      dtype='object')

In [12]:
important_columns_oceanography = [
    'wind_direction',             
    'wind_speed',                 
    'wave_height',                   
    'pressure',                   
    'air_temperature',            
    'water_temperature',          
    'Datetime',
    'Lat',
    'Lon'                 
]

important_columns_meteorology = [
    'temperature_2m',             
    'relative_humidity_2m',       
    'dew_point_2m',               
    'precipitation',              
    'pressure_msl',               
    'cloud_cover',                
    'wind_speed_10m',             
    'Datetime'
]

Handle Null Values

In [13]:
stations_depart = len(buoy_datas)
ignored_buoys = {}  # Dictionary to track ignored buoys and their reasons

for station_id, data in buoy_datas.items():
    print(f"\n🔄 Nettoyage des données pour la station {station_id}")

    marine_df = data.get("Marine")
    meteo_df = data.get("Meteo")

    if marine_df is None or meteo_df is None:
        ignored_buoys[station_id] = "Marine DataFrame ou Meteo DataFrame manquant(e)"
        print(f"⚠️ Station {station_id} ignorée: Marine DataFrame ou Meteo DataFrame manquant(e)")
        continue

    try:
        # Nettoyage des DataFrames
        cleaned_marine_df = handle_null_values(marine_df)
        cleaned_meteo_df = handle_null_values(meteo_df)
        # Vérification des colonnes importantes après nettoyage
        marine_columns_ok = all(col in cleaned_marine_df.columns for col in important_columns_oceanography)
        meteo_columns_ok = all(col in cleaned_meteo_df.columns for col in important_columns_meteorology)

        # Track which columns are missing
        missing_marine_columns = [col for col in important_columns_oceanography if col not in cleaned_marine_df.columns]
        missing_meteo_columns = [col for col in important_columns_meteorology if col not in cleaned_meteo_df.columns]

        if missing_marine_columns or missing_meteo_columns:
            ignored_buoys[station_id] = f"Colonnes manquantes: Marine: {missing_marine_columns}, Meteo: {missing_meteo_columns}"
            print(f"⚠️ Station {station_id} ignorée: Colonnes manquantes - Marine: {missing_marine_columns}, Meteo: {missing_meteo_columns}")
            continue

        # Ajouter le DataFrame nettoyé au dictionnaire des résultats
        buoy_datas[station_id]['Cleaned Marine'] = cleaned_marine_df
        buoy_datas[station_id]['Cleaned Meteo'] = cleaned_meteo_df
        print(f"✅ Nettoyage réussi pour la station {station_id} ({cleaned_marine_df.shape[0]} lignes)")

    except Exception as e:
        ignored_buoys[station_id] = f"Erreur lors du nettoyage: {e}"
        print(f"❌ Erreur lors du nettoyage pour {station_id}: {e}")

# 🔥 Suppression des stations ignorées du dictionnaire principal
for station_id in ignored_buoys:
    buoy_datas.pop(station_id, None)

len_cleaned_data = len([data for data in buoy_datas.values() if 'Cleaned Marine' in data and 'Cleaned Meteo' in data])

# Résumé final du nettoyage
print("\n📊 RÉSUMÉ DU NETTOYAGE:")
print(f"📌 Stations au départ : {stations_depart}")
print(f"✅ Stations nettoyées : {len_cleaned_data}")
print(f"🏁 Stations restantes après filtrage :")

for station_id, reason in ignored_buoys.items():
    print(f"🛑 Station {station_id} ignorée: {reason}")

print(f"\n🧹 Clés restantes dans buoy_datas après purge : {len(buoy_datas)} (attendu : {len_cleaned_data})")


🔄 Nettoyage des données pour la station 41008

Tag: orange - Nombre de lignes: 6511
Colonne 'wind_direction' Imputée par la médiane (26.37% de valeurs manquantes)
Colonne 'wind_speed' Imputée par la médiane (26.14% de valeurs manquantes)
Colonne 'wind_gust' Imputée par la médiane (26.14% de valeurs manquantes)
Colonne 'wave_height' Imputée par la médiane (49.42% de valeurs manquantes)
Colonne 'dominant_wave_period' Supprimée (plus de 55% de valeurs manquantes)
Colonne 'average_wave_period' Imputée par la médiane (49.44% de valeurs manquantes)
Colonne 'dominant_wave_direction' Imputée par la médiane (49.47% de valeurs manquantes)
Colonne 'pressure' Imputée par la médiane (0.17% de valeurs manquantes)
Colonne 'air_temperature' Imputée par la médiane (26.14% de valeurs manquantes)
Colonne 'water_temperature' Imputée par la médiane (16.66% de valeurs manquantes)
Colonne 'dewpoint' Imputée par la médiane (26.49% de valeurs manquantes)
Colonne 'visibility' Supprimée (plus de 55% de valeurs 

In [14]:
display_buoys_missing_df_counts(buoy_datas, prefix="Cleaned")


🌊 Nombre de bouées sans données 'Cleaned Marine' : 0/24

☁️ Nombre de bouées sans données 'Cleaned Meteo' : 0/24


In [15]:
# Fusion des DataFrames nettoyés
print("\n🔗 FUSION DES DONNÉES MARINE + METEO PAR STATION")

merged_success_count = 0  # Compteur de fusions réussies
total_merged_rows = 0     # Total de lignes fusionnées

for station_id, data in buoy_datas.items():
    print(f"\n🔄 Fusion des données pour la station {station_id}")

    cleaned_marine_df = data.get("Cleaned Marine")
    cleaned_meteo_df = data.get("Cleaned Meteo")

    if cleaned_marine_df is None or cleaned_meteo_df is None:
        continue

    try:
        merged_df = pd.merge(cleaned_marine_df, cleaned_meteo_df, on="Datetime", how="inner")

        if merged_df.empty:
            print(f"⚠️ Station {station_id} fusionnée, mais résultat vide après inner merge sur 'Datetime'")
        else:
            buoy_datas[station_id]["Merged"] = merged_df
            merged_success_count += 1
            total_merged_rows += len(merged_df)
            print(f"✅ Fusion réussie pour la station {station_id} ({merged_df.shape[0]} lignes)")

    except Exception as e:
        print(f"❌ Erreur lors de la fusion pour {station_id}: {e}")

# Résumé des fusions
print(f"\n📦 Fusions réussies : {merged_success_count}/{len_cleaned_data} stations")
print(f"📊 Total de lignes fusionnées : {total_merged_rows}")


🔗 FUSION DES DONNÉES MARINE + METEO PAR STATION

🔄 Fusion des données pour la station 41008
✅ Fusion réussie pour la station 41008 (1092 lignes)

🔄 Fusion des données pour la station 41044
✅ Fusion réussie pour la station 41044 (1083 lignes)

🔄 Fusion des données pour la station 41049
✅ Fusion réussie pour la station 41049 (1091 lignes)

🔄 Fusion des données pour la station 42001
✅ Fusion réussie pour la station 42001 (703 lignes)

🔄 Fusion des données pour la station 42002
✅ Fusion réussie pour la station 42002 (727 lignes)

🔄 Fusion des données pour la station 42036
✅ Fusion réussie pour la station 42036 (1082 lignes)

🔄 Fusion des données pour la station 42056
✅ Fusion réussie pour la station 42056 (1092 lignes)

🔄 Fusion des données pour la station 42058
✅ Fusion réussie pour la station 42058 (1089 lignes)

🔄 Fusion des données pour la station 44007
✅ Fusion réussie pour la station 44007 (1091 lignes)

🔄 Fusion des données pour la station 44020
✅ Fusion réussie pour la station 440

In [16]:
# Concaténation des DataFrames fusionnés
print("\n🧬 CONCATÉNATION DES DONNÉES FUSIONNÉES EN UN SEUL DATAFRAME")

final_merged_df_list = []
concat_success_count = 0
concat_total_rows = 0

for station_id, data in buoy_datas.items():
    merged_df = data.get("Merged")

    if merged_df is None:
        print(f"⚠️ Station {station_id} ignorée pour concaténation: Données fusionnées manquantes")
        continue

    try:
        final_merged_df_list.append(merged_df)
        concat_success_count += 1
        concat_total_rows += len(merged_df)
        print(f"✅ Concaténation réussie pour la station {station_id} ({len(merged_df)} lignes)")

    except Exception as e:
        print(f"❌ Erreur lors de la concaténation pour {station_id}: {e}")

# Création du DataFrame final unique
try:
    df_final = pd.concat(final_merged_df_list, ignore_index=True)
    print(f"\n🧾 DataFrame final créé avec succès ({df_final.shape[0]} lignes, {df_final.shape[1]} colonnes)")
except Exception as e:
    print(f"\n❌ Erreur lors de la création du DataFrame final: {e}")
    df_final = None

# Résumé
print(f"\n📦 Concaténations réussies : {concat_success_count}/{merged_success_count}")
print(f"📊 Total de lignes dans le DataFrame final : {concat_total_rows}")


🧬 CONCATÉNATION DES DONNÉES FUSIONNÉES EN UN SEUL DATAFRAME
✅ Concaténation réussie pour la station 41008 (1092 lignes)
✅ Concaténation réussie pour la station 41044 (1083 lignes)
✅ Concaténation réussie pour la station 41049 (1091 lignes)
✅ Concaténation réussie pour la station 42001 (703 lignes)
✅ Concaténation réussie pour la station 42002 (727 lignes)
✅ Concaténation réussie pour la station 42036 (1082 lignes)
✅ Concaténation réussie pour la station 42056 (1092 lignes)
✅ Concaténation réussie pour la station 42058 (1089 lignes)
✅ Concaténation réussie pour la station 44007 (1091 lignes)
✅ Concaténation réussie pour la station 44020 (1091 lignes)
✅ Concaténation réussie pour la station 44065 (1093 lignes)
✅ Concaténation réussie pour la station 46006 (1089 lignes)
✅ Concaténation réussie pour la station 46014 (1092 lignes)
✅ Concaténation réussie pour la station 46025 (1091 lignes)
✅ Concaténation réussie pour la station 46027 (1089 lignes)
✅ Concaténation réussie pour la station 4

In [17]:
display(df_final.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25419 entries, 0 to 25418
Data columns (total 37 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   wind_direction           25419 non-null  float64       
 1   wind_speed               25419 non-null  float64       
 2   wind_gust                25419 non-null  float64       
 3   wave_height              25419 non-null  float64       
 4   average_wave_period      25419 non-null  float64       
 5   dominant_wave_direction  23237 non-null  float64       
 6   pressure                 25419 non-null  float64       
 7   air_temperature          25419 non-null  float64       
 8   water_temperature        25419 non-null  float64       
 9   dewpoint                 25419 non-null  float64       
 10  Datetime                 25419 non-null  datetime64[ns]
 11  Lat                      25419 non-null  object        
 12  Lon                      25419 n

None

Hour Filtering

In [18]:
try:
    df_final['Datetime'] = pd.to_datetime(df_final['Datetime'])

    df_final = df_final[df_final['Datetime'].dt.minute == 0]
    # Affichage pour vérifier le résultat
    print(f"🚀 DataFrame filtrée pour ne garder que les lignes à l'heure pile: {df_final.shape[0]} lignes")

except Exception as e:
    print(f"�� Erreur lors du filtrage des lignes à l'heure pile: {e}")

🚀 DataFrame filtrée pour ne garder que les lignes à l'heure pile: 25419 lignes


In [19]:
final_datetime = pd.DataFrame(df_final[['Datetime']], dtype='datetime64[ns]')
final_datetime

,Datetime
0,2025-04-11 12:00:00
1,2025-04-11 11:00:00
2,2025-04-11 10:00:00
3,2025-04-11 09:00:00
4,2025-04-11 08:00:00
...,...
25414,2025-02-25 04:00:00
25415,2025-02-25 03:00:00
25416,2025-02-25 02:00:00
25417,2025-02-25 01:00:00


In [20]:
df_final = handle_null_values(df_final)


Tag: yellow - Nombre de lignes: 25419
Colonne 'wind_direction' non modifiée (0% de valeurs manquantes)
Colonne 'wind_speed' non modifiée (0% de valeurs manquantes)
Colonne 'wind_gust' non modifiée (0% de valeurs manquantes)
Colonne 'wave_height' non modifiée (0% de valeurs manquantes)
Colonne 'average_wave_period' non modifiée (0% de valeurs manquantes)
Colonne 'dominant_wave_direction' Imputée par la médiane (8.58% de valeurs manquantes)
Colonne 'pressure' non modifiée (0% de valeurs manquantes)
Colonne 'air_temperature' non modifiée (0% de valeurs manquantes)
Colonne 'water_temperature' non modifiée (0% de valeurs manquantes)
Colonne 'dewpoint' non modifiée (0% de valeurs manquantes)
Colonne 'Datetime' non modifiée (0% de valeurs manquantes)
Colonne 'Lat' non modifiée (0% de valeurs manquantes)
Colonne 'Lon' non modifiée (0% de valeurs manquantes)
Colonne 'Water_depth' non modifiée (0% de valeurs manquantes)
Colonne 'Station ID' non modifiée (0% de valeurs manquantes)
Colonne 'stati

In [21]:
display_row_values(df_final)

wind_direction  |  wind_speed  |  wind_gust  |  wave_height  |  average_wave_period  |  dominant_wave_direction  |  pressure  |  air_temperature  |  water_temperature  |  dewpoint  |  Datetime             |  Lat     |  Lon      |  Water_depth  |  Station ID  |  station_zone                              |  sea_temp_depth  |  Barometer_elevation  |  Anemometer_height  |  Air_temp_height  |  temperature_2m  |  relative_humidity_2m  |  dew_point_2m  |  precipitation  |  rain  |  showers  |  pressure_msl  |  surface_pressure  |  cloud_cover  |  cloud_cover_low  |  cloud_cover_mid  |  cloud_cover_high  |  visibility  |  wind_speed_10m  |  soil_temperature_0cm  |  soil_moisture_0_to_1cm  |  is_day
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [22]:
# get station id distinct values
station_ids = df_final['Station ID'].unique()
print(station_ids)

['41008' '41044' '41049' '42001' '42002' '42036' '42056' '42058' '44007'
 '44020' '44065' '46006' '46014' '46025' '46027' '46072' '46078' '46084'
 '46086' '46087' '46088' '51000' '51001' '51002']


In [23]:
df_final.columns = [col.strip() for col in df_final.columns]

In [24]:
def display_row_values(df, columns=None):
    # Si aucune colonne n'est spécifiée, afficher toutes les colonnes
    if columns is None:
        columns = df.columns.tolist()

    if isinstance(columns, str):
        columns = [columns]
    # Calculer la largeur maximale pour chaque colonne spécifiée
    column_widths = [max(df[col].astype(str).apply(len).max(), len(col)) for col in columns]

    # Afficher les noms des colonnes, en tenant compte des largeurs maximales
    column_headers = [col.ljust(column_widths[i]) for i, col in enumerate(columns)]
    print("  |  ".join(column_headers))
    print("-" * (sum(column_widths) + (len(columns) - 1) * 4))  # Ligne de séparation

    # Afficher les 10 premières valeurs sous chaque colonne spécifiée, alignées
    for i in range(min(10, len(df))):  # Affiche jusqu'à 10 lignes ou le nombre de lignes disponibles
        row_values = [str(df.iloc[i, df.columns.get_loc(col)]).ljust(column_widths[j]) for j, col in enumerate(columns)]
        print("  |  ".join(row_values))

In [25]:
# Dictionnaire de renommage des colonnes
col_to_rename = {'temperature_2m': 'T°(C°)', 
                 'relative_humidity_2m': 'Relative Humidity (%)',
                 'dew_point_2m': 'Dew Point (°C)', 
                 'precipitation': 'Precipitations (mm)',  
                 'pressure_msl':'Sea Level Pressure (hPa)', 
                 'cloud_cover_low':'Low Clouds (%)',
                 'cloud_cover_mid' : 'Middle Clouds (%)', 
                 'cloud_cover_high' : 'High Clouds (%)', 
                 'visibility' : 'Visibility (km)', 
                 'wind_direction': 'Wind Direction (°)',
                 'wind_speed': 'Wind Speed (km/h)', 
                 'wind_gust': 'Wind Gusts (km/h)',
                 'wind_speed_10m':'Wind Speed (10m)', 
                 'surface_pressure': 'Surface Pressure',
                 'wave_height': 'Wave Height (m)', 
                 'average_wave_period': 'Average Wave Period (s)',
                 'dominant_wave_direction': 'Dominant Wave Direction (°)', 
                 'pressure': 'Pressure (hPa)',
                 'air_temperature': 'Air T°', 
                 'water_temperature': 'Water T°', 
                 'Water_depth': 'Water Depth (m)', 
                 "Air_temp_height": "Air T° Height", 
                 "Anemometer_height": "Anemometer Height (m)", 
                 "station_zone": "Station Zone",
                 "Barometer_elevation": "Barometer Elevation", 
                 "sea_temp_depth" : "Sea Temperature Depth (m)"}

# Liste des colonnes à supprimer
cols_to_delete = ['soil_temperature_0cm', 'lat_buoy','lon_buoy', 'rain', 
                  'showers', 'is_day', 'soil_moisture_0_to_1cm', 
                  'cloud_cover']
	

# Renommer les colonnes d'abord
df_final = rename_columns(df_final, col_to_rename)
# Ensuite, supprimer les colonnes non désirées
df_final = drop_columns_if_exist(df_final, cols_to_delete)
try:
    df_final["Visibility (km)"].map(lambda x: x/1000)
    df_final["T°(C°)"] = round(df_final["T°(C°)"], 2)
    df_final["Wind Speed (10m)"] = round(df_final["Wind Speed (10m)"], 2)
except Exception as e:
    print(f"�� Erreur lors du traitement des colonnes :\n {e}")

# Afficher les résultats
print("\nColonnes après renommage et suppression :")
print(df_final.columns)

🔄 Colonne 'temperature_2m' renommée en 'T°(C°)'
🔄 Colonne 'relative_humidity_2m' renommée en 'Relative Humidity (%)'
🔄 Colonne 'dew_point_2m' renommée en 'Dew Point (°C)'
🔄 Colonne 'precipitation' renommée en 'Precipitations (mm)'
🔄 Colonne 'pressure_msl' renommée en 'Sea Level Pressure (hPa)'
🔄 Colonne 'cloud_cover_low' renommée en 'Low Clouds (%)'
🔄 Colonne 'cloud_cover_mid' renommée en 'Middle Clouds (%)'
🔄 Colonne 'cloud_cover_high' renommée en 'High Clouds (%)'
🔄 Colonne 'visibility' renommée en 'Visibility (km)'
🔄 Colonne 'wind_direction' renommée en 'Wind Direction (°)'
🔄 Colonne 'wind_speed' renommée en 'Wind Speed (km/h)'
🔄 Colonne 'wind_gust' renommée en 'Wind Gusts (km/h)'
🔄 Colonne 'wind_speed_10m' renommée en 'Wind Speed (10m)'
🔄 Colonne 'surface_pressure' renommée en 'Surface Pressure'
🔄 Colonne 'wave_height' renommée en 'Wave Height (m)'
🔄 Colonne 'average_wave_period' renommée en 'Average Wave Period (s)'
🔄 Colonne 'dominant_wave_direction' renommée en 'Dominant Wave Di

In [26]:
try:
    display_row_values(df_final, columns=["Visibility (km)", "T°(C°)"])
#    df_final["Visibility (km)"] = df_final["Visibility (km)"].map(lambda x: x / 1000)
    
    df_final["T°(C°)"] = df_final["T°(C°)"].round(2)
    display_row_values(df_final, ["Visibility (km)", "T°(C°)"])
except Exception as e:
    print(f"�� Erreur : {e}")

Visibility (km)  |  T°(C°)
-------------------------
25500.0          |  18.0  
25900.0          |  18.4  
19900.0          |  18.95 
18300.0          |  18.9  
18600.0          |  20.2  
13900.0          |  20.1  
13900.0          |  20.0  
14000.0          |  20.1  
14800.0          |  20.0  
14400.0          |  19.9  
Visibility (km)  |  T°(C°)
-------------------------
25500.0          |  18.0  
25900.0          |  18.4  
19900.0          |  18.95 
18300.0          |  18.9  
18600.0          |  20.2  
13900.0          |  20.1  
13900.0          |  20.0  
14000.0          |  20.1  
14800.0          |  20.0  
14400.0          |  19.9  


In [27]:
display_row_values(df_final)

Wind Direction (°)  |  Wind Speed (km/h)  |  Wind Gusts (km/h)  |  Wave Height (m)  |  Average Wave Period (s)  |  Dominant Wave Direction (°)  |  Pressure (hPa)  |  Air T°  |  Water T°  |  dewpoint  |  Datetime             |  Lat     |  Lon      |  Water Depth (m)  |  Station ID  |  Station Zone                              |  Sea Temperature Depth (m)  |  Barometer Elevation  |  Anemometer Height (m)  |  Air T° Height  |  T°(C°)  |  Relative Humidity (%)  |  Dew Point (°C)  |  Precipitations (mm)  |  Sea Level Pressure (hPa)  |  Surface Pressure  |  Low Clouds (%)  |  Middle Clouds (%)  |  High Clouds (%)  |  Visibility (km)  |  Wind Speed (10m)
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [28]:
try:
    df_final[['Lat', 'Lon']] = df_final.apply(
        lambda row: pd.Series(convert_coordinates(row['Lat'], row['Lon'])),
        axis=1
    )
except Exception as e:
    print(f"❌ Erreur lors de la conversion des coordonnées : {e}")


In [29]:
#  T°(C°) = (Air T° + T°(C°))/2 et virer Air T°
try:
    df_final['T°(C°)'] = (df_final['Air T°'] + df_final['T°(C°)']) / 2
    df_final.drop(columns=['Air T°'], inplace=True)
except:
    pass

# Sea Level Pressure (hPa) = (Sea Level Pressure (hPa) +Surface Pressure)/2 et virer Surface Pressure

try:
    df_final['Sea Level Pressure (hPa)'] = (df_final['Sea Level Pressure (hPa)'] + df_final['Surface Pressure']) / 2
    df_final.drop(columns=['Surface Pressure'], inplace=True)
except:
    pass

# virer le m dans Water Depth avec regex lambda et passer la colonne en float
df_final['Water Depth (m)'] = df_final['Water Depth (m)'].apply(lambda x: re.sub(r'\D', '', str(x)).strip())
df_final['Water Depth (m)'].astype(float)

0          16.0
1          16.0
2          16.0
3          16.0
4          16.0
          ...  
25414    4979.0
25415    4979.0
25416    4979.0
25417    4979.0
25418    4979.0
Name: Water Depth (m), Length: 25419, dtype: float64

In [30]:
display_row_values(df_final)

Wind Direction (°)  |  Wind Speed (km/h)  |  Wind Gusts (km/h)  |  Wave Height (m)  |  Average Wave Period (s)  |  Dominant Wave Direction (°)  |  Pressure (hPa)  |  Water T°  |  dewpoint  |  Datetime             |  Lat    |  Lon      |  Water Depth (m)  |  Station ID  |  Station Zone                              |  Sea Temperature Depth (m)  |  Barometer Elevation  |  Anemometer Height (m)  |  Air T° Height  |  T°(C°)                  |  Relative Humidity (%)  |  Dew Point (°C)  |  Precipitations (mm)  |  Sea Level Pressure (hPa)  |  Low Clouds (%)  |  Middle Clouds (%)  |  High Clouds (%)  |  Visibility (km)  |  Wind Speed (10m)
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Check truth about Wind Speed Using another API call

In [31]:
df_42058 = df_final[df_final['Station ID'] == "42058"]
df_42058.columns

Index(['Wind Direction (°)', 'Wind Speed (km/h)', 'Wind Gusts (km/h)',
       'Wave Height (m)', 'Average Wave Period (s)',
       'Dominant Wave Direction (°)', 'Pressure (hPa)', 'Water T°', 'dewpoint',
       'Datetime', 'Lat', 'Lon', 'Water Depth (m)', 'Station ID',
       'Station Zone', 'Sea Temperature Depth (m)', 'Barometer Elevation',
       'Anemometer Height (m)', 'Air T° Height', 'T°(C°)',
       'Relative Humidity (%)', 'Dew Point (°C)', 'Precipitations (mm)',
       'Sea Level Pressure (hPa)', 'Low Clouds (%)', 'Middle Clouds (%)',
       'High Clouds (%)', 'Visibility (km)', 'Wind Speed (10m)'],
      dtype='object')

In [32]:
# ---- Chargement de la clé API ----
vc_api_key_path = r"c:\Credentials\visual_crossing_weather_api.json"
with open(vc_api_key_path, 'r') as file:
    content = json.load(file)
    vc_api_key = content["api_key"]

# ---- Extraire les coordonnées depuis la première ligne du DataFrame ----
lat_42058, lon_42058 = None, None
if not df_42058.empty:
    first_row = df_42058.iloc[0]
    lat_42058, lon_42058 = first_row["Lat"], first_row["Lon"]

# ---- Définir les dates pour la requête ----
today = datetime.now().strftime("%Y-%m-%d")
last_month = (datetime.now() - timedelta(days=31)).strftime("%Y-%m-%d")

# ---- Créer le dossier de cache si nécessaire ----
cache_dir = "api_call_files"
os.makedirs(cache_dir, exist_ok=True)

# ---- Définir le fichier cache selon la position ----
cache_file = os.path.join(cache_dir, f"vc_meteo_{lat_42058}_{lon_42058}.csv")

# ---- Vérifier si un cache récent existe (moins de 24h) ----
use_cache = False
if os.path.exists(cache_file):
    last_modified = datetime.fromtimestamp(os.path.getmtime(cache_file))
    if datetime.now() - last_modified < timedelta(hours=24):
        print(f"📦 Cache détecté ({cache_file}), modifié le {last_modified.strftime('%Y-%m-%d %H:%M:%S')}")
        vc_meteo_df = pd.read_csv(cache_file)
        print("✅ Données météo rechargées depuis le cache.")
        use_cache = True
    else:
        print(f"⚠️ Cache trouvé mais périmé (plus de 24h) → nouvelle requête API.")

# ---- Appel API si pas de cache valide ----
if not use_cache and lat_42058 is not None and lon_42058 is not None:
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{lat_42058},{lon_42058}/{last_month}/{today}?unitGroup=metric&key={vc_api_key}&contentType=json"
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            vc_meteo_data = response.json()
            print("🌍 Données météo récupérées depuis l'API Visual Crossing.")

            # ---- Extraire les données journalières et sauvegarder ----
            if "days" in vc_meteo_data:
                vc_meteo_df = pd.json_normalize(vc_meteo_data["days"])
                vc_meteo_df.to_csv(cache_file, index=False)
                print(f"💾 Données sauvegardées dans le cache : {cache_file}")
            else:
                print("⚠️ Le champ 'days' est absent de la réponse API.")
        else:
            print(f"❌ Échec de l’appel API — code de statut : {response.status_code}")
    except Exception as e:
        print(f"❌ Exception levée lors de la requête API : {e}")

📦 Cache détecté (api_call_files\vc_meteo_14.51_-75.15.csv), modifié le 2025-04-10 16:45:13
✅ Données météo rechargées depuis le cache.


In [33]:
# 📁 Charger les données du CSV Visual Crossing
vc_csv_path = f"api_call_files/vc_meteo_{lat_42058}_{lon_42058}.csv"
df_vc_meteo = pd.read_csv(vc_csv_path)
df_vc_meteo.head()

,datetime,datetimeEpoch,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,sunriseEpoch,sunset,sunsetEpoch,moonphase,conditions,description,icon,stations,source,hours
0,2025-03-10,1741582800,27.4,26.9,27.3,30.3,29.3,30.0,23.2,78.5,...,1741605083,18:10:23,1741648223,0.37,Clear,Clear conditions throughout the day.,clear-day,['42058_maritime'],obs,"[{'datetime': '00:00:00', 'datetimeEpoch': 174..."
1,2025-03-11,1741669200,27.4,26.9,27.2,30.2,29.1,29.7,22.9,77.5,...,1741691442,18:10:31,1741734631,0.41,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,['42058_maritime'],obs,"[{'datetime': '00:00:00', 'datetimeEpoch': 174..."
2,2025-03-12,1741755600,27.3,26.5,27.0,29.9,26.5,28.9,22.4,76.1,...,1741777802,18:10:40,1741821040,0.44,Clear,Clear conditions throughout the day.,clear-day,['42058_maritime'],obs,"[{'datetime': '00:00:00', 'datetimeEpoch': 174..."
3,2025-03-13,1741842000,27.4,26.9,27.1,30.3,29.1,29.7,22.9,77.5,...,1741864161,18:10:48,1741907448,0.48,Clear,Clear conditions throughout the day.,clear-day,"['42058_maritime', 'remote']",obs,"[{'datetime': '00:00:00', 'datetimeEpoch': 174..."
4,2025-03-14,1741928400,27.4,26.5,27.2,30.2,26.5,29.6,22.8,77.1,...,1741950520,18:10:56,1741993856,0.50,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"['42058_maritime', 'remote']",obs,"[{'datetime': '00:00:00', 'datetimeEpoch': 174..."


In [34]:
# 🧼 Nettoyage et transformation
import ast  # Permet de convertir les strings représentant des listes en objets Python
all_hours = []

# Parcours de chaque ligne du fichier météo
for i, row in df_vc_meteo.iterrows():
    try:
        # Convertir la colonne "hours" en liste de dictionnaires
        hours_list = ast.literal_eval(row['hours'])

        # Pour chaque heure dans la journée, on ajoute une entrée au tableau final
        for hour_data in hours_list:
            hour_data['Date'] = row['datetime']  # On ajoute aussi la date du jour correspondant
            all_hours.append(hour_data)

    except Exception as e:
        print(f"Erreur parsing ligne {i}: {e}")  # En cas d'erreur de parsing (souvent problème de format)

# ✅ On transforme la liste aplanie en DataFrame
df_vc_flat = pd.DataFrame(all_hours)

# 🕒 Conversion de l'heure/temps
df_vc_flat["Datetime"] = pd.to_datetime(df_vc_flat["datetimeEpoch"], unit="s").dt.strftime("%Y-%m-%d-%H")

# 🗓️ Filtrer sur les 30 derniers jours
today = datetime.now()
thirty_days_ago = today - timedelta(days=30)

today_str = today.strftime("%Y-%m-%d")
thirty_days_ago_str = thirty_days_ago.strftime("%Y-%m-%d")

# On filtre le DataFrame pour ne garder que les lignes entre ces deux dates
df_vc_last_month = df_vc_flat[
    (df_vc_flat['Date'] >= thirty_days_ago_str) & 
    (df_vc_flat['Date'] <= today_str)]

In [35]:
display_row_values(df_vc_last_month)

datetime  |  datetimeEpoch  |  temp  |  feelslike  |  humidity  |  dew   |  precip  |  precipprob  |  snow  |  snowdepth  |  preciptype  |  windgust  |  windspeed  |  winddir  |  pressure  |  visibility  |  cloudcover  |  solarradiation  |  solarenergy  |  uvindex  |  severerisk  |  conditions              |  icon                 |  stations            |  source  |  Date        |  Datetime     
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
00:00:00  |  1741755600     |  27.0  |  29.6       |  79.73     |  23.2  |  0.0     |  0.0         |  0.0   |  0.0        |  None        |  29.9      |  21.6       |  60.0     |  1013.4    |  24.1        |  29.4        |  0.0     

In [36]:
# Correct syntaxe pour filtrer les colonnes spécifiques
df_vc_last_month = df_vc_last_month[["Datetime", "temp", "humidity", "precip", "dew", "windgust", "windspeed", "winddir", "pressure", "visibility"]]

for col in df_vc_last_month.columns:
    if not col == "Datetime":
        rename_columns(df_vc_last_month, {col: f"VC_{col}"})

🔄 Colonne 'temp' renommée en 'VC_temp'
✅ Colonnes renommées : {'temp': 'VC_temp'}
🔄 Colonne 'humidity' renommée en 'VC_humidity'
✅ Colonnes renommées : {'humidity': 'VC_humidity'}
🔄 Colonne 'precip' renommée en 'VC_precip'
✅ Colonnes renommées : {'precip': 'VC_precip'}
🔄 Colonne 'dew' renommée en 'VC_dew'
✅ Colonnes renommées : {'dew': 'VC_dew'}
🔄 Colonne 'windgust' renommée en 'VC_windgust'
✅ Colonnes renommées : {'windgust': 'VC_windgust'}
🔄 Colonne 'windspeed' renommée en 'VC_windspeed'
✅ Colonnes renommées : {'windspeed': 'VC_windspeed'}
🔄 Colonne 'winddir' renommée en 'VC_winddir'
✅ Colonnes renommées : {'winddir': 'VC_winddir'}
🔄 Colonne 'pressure' renommée en 'VC_pressure'
✅ Colonnes renommées : {'pressure': 'VC_pressure'}
🔄 Colonne 'visibility' renommée en 'VC_visibility'
✅ Colonnes renommées : {'visibility': 'VC_visibility'}


In [37]:
# Convert 'Datetime' column to datetime type in df_vc_last_month
df_vc_last_month['Datetime'] = pd.to_datetime(df_vc_last_month['Datetime'], errors='coerce')


# Check if the Datetime column is correctly converted
print(df_42058['Datetime'].dtype)
print(df_vc_last_month['Datetime'].dtype)

datetime64[ns]
datetime64[ns]


In [38]:
# Now perform the merge
df_compare = pd.merge(df_42058, df_vc_last_month, on="Datetime", how="inner")

df_compare.columns

Index(['Wind Direction (°)', 'Wind Speed (km/h)', 'Wind Gusts (km/h)',
       'Wave Height (m)', 'Average Wave Period (s)',
       'Dominant Wave Direction (°)', 'Pressure (hPa)', 'Water T°', 'dewpoint',
       'Datetime', 'Lat', 'Lon', 'Water Depth (m)', 'Station ID',
       'Station Zone', 'Sea Temperature Depth (m)', 'Barometer Elevation',
       'Anemometer Height (m)', 'Air T° Height', 'T°(C°)',
       'Relative Humidity (%)', 'Dew Point (°C)', 'Precipitations (mm)',
       'Sea Level Pressure (hPa)', 'Low Clouds (%)', 'Middle Clouds (%)',
       'High Clouds (%)', 'Visibility (km)', 'Wind Speed (10m)', 'VC_temp',
       'VC_humidity', 'VC_precip', 'VC_dew', 'VC_windgust', 'VC_windspeed',
       'VC_winddir', 'VC_pressure', 'VC_visibility'],
      dtype='object')

In [39]:
try:
    df_windspeed_compare = df_compare[['Wind Speed (km/h)', 'Wind Speed (10m)','VC_windspeed']]

    df_pressure_compare = df_compare[['Sea Level Pressure (hPa)','Pressure (hPa)', 'VC_pressure']]
    df_dew_compare = df_compare[['dewpoint','Dew Point (°C)', 'VC_dew']]
except Exception as e:
    print(e)

Comparaison Wind Speed

In [40]:
df_windspeed_compare.head(10)

,Wind Speed (km/h),Wind Speed (10m),VC_windspeed
0,7.0,25.570000,29.9
1,8.0,25.930000,30.6
2,7.0,26.840000,29.5
3,7.0,26.120001,29.2
4,7.0,25.070000,29.2
5,7.0,24.360001,29.9
6,7.0,21.070000,28.1
7,6.0,17.299999,25.6
8,7.0,17.340000,24.5
9,7.0,18.730000,24.5


In [41]:
wind_col_to_delete = ['Wind Speed (km/h)', 'Anemometer Height (m)']
df_final = drop_columns_if_exist(df_final, wind_col_to_delete)

 Nombre initial de colonnes: 29
Colonne 'Wind Speed (km/h)' Supprimée
Colonne 'Anemometer Height (m)' Supprimée
 Nombre final de colonnes: 27


Comparaison Pressure

In [42]:
df_pressure_compare.head()

,Sea Level Pressure (hPa),Pressure (hPa),VC_pressure
0,1013.599976,1012.8,1013.0
1,1013.400024,1013.0,1013.0
2,1013.000000,1012.6,1013.0
3,1012.200012,1012.3,1013.0
4,1011.799988,1011.5,1012.0


In [43]:
df_final = drop_columns_if_exist(df_final, ['Pressure (hPa)'])

 Nombre initial de colonnes: 27
Colonne 'Pressure (hPa)' Supprimée
 Nombre final de colonnes: 26


In [44]:
df_dew_compare.head(20)

,dewpoint,Dew Point (°C),VC_dew
0,25.3,24.360001,24.2
1,24.9,24.510000,24.4
2,25.0,24.500000,24.3
3,25.1,24.299999,24.1
4,25.0,24.500000,24.2
5,24.9,24.200001,24.2
6,25.0,24.190001,24.2
7,25.0,24.500000,24.0
8,24.8,24.500000,23.8
9,24.8,24.549999,23.7


In [45]:
try:

    # Calcul des distances absolues entre chaque paire de mesures pour chaque ligne
    # Ces distances nous permettent de savoir quelle mesure est la plus proche des autres
    df_compare['dist_dewpoint_DewPoint'] = np.abs(df_compare['dewpoint'] - df_compare['Dew Point (°C)'])
    df_compare['dist_dewpoint_VC'] = np.abs(df_compare['dewpoint'] - df_compare['VC_dew'])
    df_compare['dist_DewPoint_VC'] = np.abs(df_compare['Dew Point (°C)'] - df_compare['VC_dew'])

    # Pour chaque ligne, on détermine quelle mesure est la plus proche des deux autres :
    # - Si 'dewpoint' est plus proche des autres mesures que 'Dew Point (°C)' et 'VC_dew', alors 'dewpoint' est marqué comme plus proche.
    df_compare['dewpoint_closer'] = (df_compare['dist_dewpoint_VC'] < df_compare['dist_dewpoint_DewPoint']) & (df_compare['dist_dewpoint_VC'] < df_compare['dist_DewPoint_VC'])

    # - Si 'Dew Point (°C)' est plus proche des autres mesures que 'dewpoint' et 'VC_dew', alors 'Dew Point (°C)' est marqué comme plus proche.
    df_compare['DewPoint_closer'] = (df_compare['dist_dewpoint_DewPoint'] < df_compare['dist_dewpoint_VC']) & (df_compare['dist_dewpoint_DewPoint'] < df_compare['dist_DewPoint_VC'])

    # - Si 'VC_dew' est plus proche des autres mesures que 'dewpoint' et 'Dew Point (°C)', alors 'VC_dew' est marqué comme plus proche.
    df_compare['VC_closer'] = (df_compare['dist_DewPoint_VC'] < df_compare['dist_dewpoint_VC']) & (df_compare['dist_DewPoint_VC'] < df_compare['dist_dewpoint_DewPoint'])

    # Calculer les probabilités que chaque mesure soit la plus proche des autres sur l'ensemble des lignes :
    # La probabilité est simplement la proportion de fois où une mesure a été plus proche des autres.
    prob_dewpoint_closer = df_compare['dewpoint_closer'].mean().round(3)
    prob_dewpoint_c_closer = df_compare['DewPoint_closer'].mean().round(3)
    prob_vc_closer = df_compare['VC_closer'].mean().round(3)

    # Afficher les résultats
    # Ces résultats indiquent la probabilité que chaque mesure soit la plus proche des autres sur toutes les lignes de données
    print(f"Probability that 'dewpoint' is closer to the truth: {prob_dewpoint_closer}")
    print(f"Probability that 'Dew Point (°C)' is closer to the truth: {prob_dewpoint_c_closer}")
    print(f"Probability that 'VC_dew' is closer to the truth: {prob_vc_closer}")
    
except Exception as e:
    print(e)

Probability that 'dewpoint' is closer to the truth: 0.814
Probability that 'Dew Point (°C)' is closer to the truth: 0.091
Probability that 'VC_dew' is closer to the truth: 0.087


In [46]:
try:
    df_final['Dew Point (°C)'] = df_final['dewpoint']
    df_final = drop_columns_if_exist(df_final, ['dewpoint'])
except Exception as e:
    print(e)

 Nombre initial de colonnes: 26
Colonne 'dewpoint' Supprimée
 Nombre final de colonnes: 25


In [47]:
display_row_values(df_final)

Wind Direction (°)  |  Wind Gusts (km/h)  |  Wave Height (m)  |  Average Wave Period (s)  |  Dominant Wave Direction (°)  |  Water T°  |  Datetime             |  Lat    |  Lon      |  Water Depth (m)  |  Station ID  |  Station Zone                              |  Sea Temperature Depth (m)  |  Barometer Elevation  |  Air T° Height  |  T°(C°)                  |  Relative Humidity (%)  |  Dew Point (°C)  |  Precipitations (mm)  |  Sea Level Pressure (hPa)  |  Low Clouds (%)  |  Middle Clouds (%)  |  High Clouds (%)  |  Visibility (km)  |  Wind Speed (10m)
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [48]:
df_final = add_hour_day_period_and_month(df_final)
print(df_final.columns)

Index(['Wind Direction (°)', 'Wind Gusts (km/h)', 'Wave Height (m)',
       'Average Wave Period (s)', 'Dominant Wave Direction (°)', 'Water T°',
       'Datetime', 'Lat', 'Lon', 'Water Depth (m)', 'Station ID',
       'Station Zone', 'Sea Temperature Depth (m)', 'Barometer Elevation',
       'Air T° Height', 'T°(C°)', 'Relative Humidity (%)', 'Dew Point (°C)',
       'Precipitations (mm)', 'Sea Level Pressure (hPa)', 'Low Clouds (%)',
       'Middle Clouds (%)', 'High Clouds (%)', 'Visibility (km)',
       'Wind Speed (10m)', 'DayPeriod', 'Month', 'Hour'],
      dtype='object')


In [49]:
display_row_values(df_final)

Wind Direction (°)  |  Wind Gusts (km/h)  |  Wave Height (m)  |  Average Wave Period (s)  |  Dominant Wave Direction (°)  |  Water T°  |  Datetime             |  Lat    |  Lon      |  Water Depth (m)  |  Station ID  |  Station Zone                              |  Sea Temperature Depth (m)  |  Barometer Elevation  |  Air T° Height  |  T°(C°)                  |  Relative Humidity (%)  |  Dew Point (°C)  |  Precipitations (mm)  |  Sea Level Pressure (hPa)  |  Low Clouds (%)  |  Middle Clouds (%)  |  High Clouds (%)  |  Visibility (km)  |  Wind Speed (10m)  |  DayPeriod  |  Month  |  Hour
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [50]:
display(df_final.info())
display(df_final.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25419 entries, 0 to 25418
Data columns (total 28 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Wind Direction (°)           25419 non-null  float64       
 1   Wind Gusts (km/h)            25419 non-null  float64       
 2   Wave Height (m)              25419 non-null  float64       
 3   Average Wave Period (s)      25419 non-null  float64       
 4   Dominant Wave Direction (°)  25419 non-null  float64       
 5   Water T°                     25419 non-null  float64       
 6   Datetime                     25419 non-null  datetime64[ns]
 7   Lat                          25419 non-null  float64       
 8   Lon                          25419 non-null  float64       
 9   Water Depth (m)              25419 non-null  object        
 10  Station ID                   25419 non-null  object        
 11  Station Zone                 25419 non-nu

None

Index(['Wind Direction (°)', 'Wind Gusts (km/h)', 'Wave Height (m)',
       'Average Wave Period (s)', 'Dominant Wave Direction (°)', 'Water T°',
       'Datetime', 'Lat', 'Lon', 'Water Depth (m)', 'Station ID',
       'Station Zone', 'Sea Temperature Depth (m)', 'Barometer Elevation',
       'Air T° Height', 'T°(C°)', 'Relative Humidity (%)', 'Dew Point (°C)',
       'Precipitations (mm)', 'Sea Level Pressure (hPa)', 'Low Clouds (%)',
       'Middle Clouds (%)', 'High Clouds (%)', 'Visibility (km)',
       'Wind Speed (10m)', 'DayPeriod', 'Month', 'Hour'],
      dtype='object')

In [51]:
# Charger les informations de connexion MySQL
path_to_mysql_creds = r"c:\Credentials\mysql_creds.json"
with open(path_to_mysql_creds, 'r') as file:
    content = json.load(file)
    mysql_user = content["user"]
    password = content["password"]
    host = content["host"]
    port = content["port"]
    database = "oceanography_data_analysis"
    silver_table = "silver table"

# Connexion à MySQL avec SQLModel
engine = create_engine(f"mysql+mysqlconnector://{mysql_user}:{password}@{host}:{port}/{database}", isolation_level='AUTOCOMMIT')
Session = sessionmaker(bind=engine)
session = Session()

In [52]:
df_final.columns

Index(['Wind Direction (°)', 'Wind Gusts (km/h)', 'Wave Height (m)',
       'Average Wave Period (s)', 'Dominant Wave Direction (°)', 'Water T°',
       'Datetime', 'Lat', 'Lon', 'Water Depth (m)', 'Station ID',
       'Station Zone', 'Sea Temperature Depth (m)', 'Barometer Elevation',
       'Air T° Height', 'T°(C°)', 'Relative Humidity (%)', 'Dew Point (°C)',
       'Precipitations (mm)', 'Sea Level Pressure (hPa)', 'Low Clouds (%)',
       'Middle Clouds (%)', 'High Clouds (%)', 'Visibility (km)',
       'Wind Speed (10m)', 'DayPeriod', 'Month', 'Hour'],
      dtype='object')

In [53]:
display_row_values(df_final)

Wind Direction (°)  |  Wind Gusts (km/h)  |  Wave Height (m)  |  Average Wave Period (s)  |  Dominant Wave Direction (°)  |  Water T°  |  Datetime             |  Lat    |  Lon      |  Water Depth (m)  |  Station ID  |  Station Zone                              |  Sea Temperature Depth (m)  |  Barometer Elevation  |  Air T° Height  |  T°(C°)                  |  Relative Humidity (%)  |  Dew Point (°C)  |  Precipitations (mm)  |  Sea Level Pressure (hPa)  |  Low Clouds (%)  |  Middle Clouds (%)  |  High Clouds (%)  |  Visibility (km)  |  Wind Speed (10m)  |  DayPeriod  |  Month  |  Hour
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [54]:
df_final.columns

Index(['Wind Direction (°)', 'Wind Gusts (km/h)', 'Wave Height (m)',
       'Average Wave Period (s)', 'Dominant Wave Direction (°)', 'Water T°',
       'Datetime', 'Lat', 'Lon', 'Water Depth (m)', 'Station ID',
       'Station Zone', 'Sea Temperature Depth (m)', 'Barometer Elevation',
       'Air T° Height', 'T°(C°)', 'Relative Humidity (%)', 'Dew Point (°C)',
       'Precipitations (mm)', 'Sea Level Pressure (hPa)', 'Low Clouds (%)',
       'Middle Clouds (%)', 'High Clouds (%)', 'Visibility (km)',
       'Wind Speed (10m)', 'DayPeriod', 'Month', 'Hour'],
      dtype='object')

In [55]:
try:
    df_final.rename(columns={'Air T° Height': 'Air T° Height (m)',
                             'Barometer Elevation': 'Barometer Elevation (m)',
                             'Water T°': 'Water T° (°C)'}, inplace=True)

    print("Column 'Air T° Height' renamed to 'Air T° Height (m)'")

    print("Column 'Barometer Elevation' renamed to 'Barometer Elevation (m)'")
    print("Column 'Water T°' renamed to 'Water T° (°C)'")
    df_final['T°(C°)'] = df_final['T°(C°)'].round(2)

    print("Column 'T°(C°)' rounded to 2 decimal places")
except Exception as e:
    print(e)
    
display_row_values(df_final)

Column 'Air T° Height' renamed to 'Air T° Height (m)'
Column 'Barometer Elevation' renamed to 'Barometer Elevation (m)'
Column 'Water T°' renamed to 'Water T° (°C)'
Column 'T°(C°)' rounded to 2 decimal places
Wind Direction (°)  |  Wind Gusts (km/h)  |  Wave Height (m)  |  Average Wave Period (s)  |  Dominant Wave Direction (°)  |  Water T° (°C)  |  Datetime             |  Lat    |  Lon      |  Water Depth (m)  |  Station ID  |  Station Zone                              |  Sea Temperature Depth (m)  |  Barometer Elevation (m)  |  Air T° Height (m)  |  T°(C°)  |  Relative Humidity (%)  |  Dew Point (°C)  |  Precipitations (mm)  |  Sea Level Pressure (hPa)  |  Low Clouds (%)  |  Middle Clouds (%)  |  High Clouds (%)  |  Visibility (km)  |  Wind Speed (10m)  |  DayPeriod  |  Month  |  Hour
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [56]:
# Convertir 'Station ID' en chaîne de caractères et concaténer avec la date sous forme de 'YYYYMMDDHH'
df_final['ID Unique'] = df_final['Station ID'].astype(str) + df_final['Datetime'].dt.strftime('%Y%m%d%H')

# Obtenir les paires uniques de 'ID Unique' et 'Station ID'
df_uniques_and_station_id = df_final[['ID Unique', 'Station ID']].drop_duplicates()
# Réorganiser les colonnes pour que 'ID Unique' soit en premier
cols = ['ID Unique'] + [col for col in df_final.columns if col != 'ID Unique']
df_final = df_final[cols]

display_row_values(df_final)

ID Unique        |  Wind Direction (°)  |  Wind Gusts (km/h)  |  Wave Height (m)  |  Average Wave Period (s)  |  Dominant Wave Direction (°)  |  Water T° (°C)  |  Datetime             |  Lat    |  Lon      |  Water Depth (m)  |  Station ID  |  Station Zone                              |  Sea Temperature Depth (m)  |  Barometer Elevation (m)  |  Air T° Height (m)  |  T°(C°)  |  Relative Humidity (%)  |  Dew Point (°C)  |  Precipitations (mm)  |  Sea Level Pressure (hPa)  |  Low Clouds (%)  |  Middle Clouds (%)  |  High Clouds (%)  |  Visibility (km)  |  Wind Speed (10m)  |  DayPeriod  |  Month  |  Hour
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
create_table_in_mysql(df=df_final, engine=engine, table_name=silver_table)
insert_new_rows(df=df_final, engine=engine, silver_table=silver_table, ref_column='ID Unique')

✅ Table 'silver table' créée avec succès dans MySQL.


Total DF code

In [63]:
##################################### DimStation ##################################################################################################
df_station = df_final[['Station ID', 'Station Zone', 'Lat', 'Lon']].drop_duplicates()
df_station = df_station[['Station ID', 'Station Zone', 'Lat', 'Lon']]

print(f"\nNul count in df_station:\n{df_station.isnull().sum()}")

###################################### DimTime ##################################################################################################
df_final['Year'] = df_final['Datetime'].dt.year
df_final['Day'] = df_final['Datetime'].dt.day
df_final['DayOfWeek'] = df_final['Datetime'].dt.weekday
df_final['Hour'] = df_final['Datetime'].dt.hour
df_time = df_final[['Year', 'Month', 'Day', 'Hour', 'DayOfWeek', 'DayPeriod']].drop_duplicates()
df_time['Date ID'] = df_time.apply(lambda row: int(f"{row.Year:04d}{row.Month:02d}{row.Day:02d}{row.Hour:02d}"), axis=1)
df_time = df_time[['Date ID', 'Year', 'Month', 'Day', 'Hour', 'DayOfWeek', 'DayPeriod']]

print(f"\nNull count in df_time:\n{df_time.isnull().sum()}")

########################################## Facts Meteo #########################################################################

df_facts_meteo = df_final[['ID Unique', 'T°(C°)', 'Relative Humidity (%)', 'Dew Point (°C)',
    'Precipitations (mm)', 'Sea Level Pressure (hPa)', 'Low Clouds (%)',
    'Middle Clouds (%)', 'High Clouds (%)', 'Visibility (km)', 'Wind Speed (10m)',
    'Wind Direction (°)', 'Wind Gusts (km/h)', 'Barometer Elevation (m)',
    'Air T° Height (m)', 'Station ID', 'Year', 'Month', 'Day', 'Hour'
]].copy()
df_facts_meteo['Date ID'] = df_facts_meteo.apply(lambda row: int(f"{row.Year:04d}{row.Month:02d}{row.Day:02d}{row.Hour:02d}"), axis=1)

# Réarrangement pour mettre les clés étrangères à la fin
df_facts_meteo = df_facts_meteo[[
    'ID Unique',  # Clé unique en premier
    'T°(C°)', 'Relative Humidity (%)', 'Dew Point (°C)', 'Precipitations (mm)',
    'Sea Level Pressure (hPa)', 'Low Clouds (%)', 'Middle Clouds (%)', 'High Clouds (%)',
    'Visibility (km)', 'Wind Speed (10m)', 'Wind Direction (°)', 'Wind Gusts (km/h)',
    'Barometer Elevation (m)', 'Air T° Height (m)', 'Date ID', 'Station ID'
]]  # Clés étrangères à la fin

# print nul count in the df
print(f"\nNull count in df_facts_meteo:\n{df_facts_meteo.isnull().sum()}")

########################################## Facts Ocean #########################################################################

df_facts_ocean = df_final[[
    'ID Unique', 
    'Wave Height (m)', 'Average Wave Period (s)', 'Dominant Wave Direction (°)',
    'Water T° (°C)', 'Water Depth (m)', 'Sea Temperature Depth (m)', 'Station ID', 'Year', 'Month', 'Day', 'Hour'
]].copy()
df_facts_ocean['Date ID'] = df_facts_ocean.apply(lambda row: int(f"{row.Year:04d}{row.Month:02d}{row.Day:02d}{row.Hour:02d}"), axis=1)

df_facts_ocean = df_facts_ocean[[
    'ID Unique',  # Clé unique en premier
    'Wave Height (m)', 'Average Wave Period (s)', 'Dominant Wave Direction (°)',
    'Water T° (°C)', 'Water Depth (m)', 'Sea Temperature Depth (m)', 'Date ID', 'Station ID'
]]  # Clés étrangères à la fin

# print nul count in the df
print(f"\nNull count in df_facts_ocean:\n{df_facts_ocean.isnull().sum()}")


Nul count in df_station:
Station ID      0
Station Zone    0
Lat             0
Lon             0
dtype: int64

Null count in df_time:
Date ID      0
Year         0
Month        0
Day          0
Hour         0
DayOfWeek    0
DayPeriod    0
dtype: int64

Null count in df_facts_meteo:
ID Unique                   0
T°(C°)                      0
Relative Humidity (%)       0
Dew Point (°C)              0
Precipitations (mm)         0
Sea Level Pressure (hPa)    0
Low Clouds (%)              0
Middle Clouds (%)           0
High Clouds (%)             0
Visibility (km)             0
Wind Speed (10m)            0
Wind Direction (°)          0
Wind Gusts (km/h)           0
Barometer Elevation (m)     0
Air T° Height (m)           0
Date ID                     0
Station ID                  0
dtype: int64

Null count in df_facts_ocean:
ID Unique                      0
Wave Height (m)                0
Average Wave Period (s)        0
Dominant Wave Direction (°)    0
Water T° (°C)                 

In [ ]:
# Vérifier les doublons
print(f"Doublons dans df_station : {df_station.duplicated().sum()}")
print(f"Doublons dans df_time : {df_time.duplicated().sum()}")
print(f"Doublons dans df_facts_meteo : {df_facts_meteo.duplicated().sum()}")
print(f"Doublons dans df_facts_ocean : {df_facts_ocean.duplicated().sum()}")


# Vérifier si les clés étrangères existent dans les dimensions
print(f"Stations ID dans df_facts_meteo qui ne sont pas dans df_station : {df_facts_meteo[~df_facts_meteo['Station ID'].isin(df_station['Station ID'])].shape[0]}")
print(f"Date ID dans df_facts_meteo qui ne sont pas dans df_time : {df_facts_meteo[~df_facts_meteo['Date ID'].isin(df_time['Date ID'])].shape[0]}")

print(f"Stations ID dans df_facts_ocean qui ne sont pas dans df_station : {df_facts_ocean[~df_facts_ocean['Station ID'].isin(df_station['Station ID'])].shape[0]}")
print(f"Date ID dans df_facts_ocean qui ne sont pas dans df_time : {df_facts_ocean[~df_facts_ocean['Date ID'].isin(df_time['Date ID'])].shape[0]}")

# Vérification des valeurs de base
print(f"\nDescription des données de df_station : \n{df_station.describe()}")
print(f"\nDescription des données de df_time : \n{df_time.describe()}")
print(f"\nDescription des données de df_facts_meteo : \n{df_facts_meteo.describe()}")
print(f"\nDescription des données de df_facts_ocean : \n{df_facts_ocean.describe()}")

In [ ]:
# check unique values
print(f"{df_station.shape[0]}\n\n{df_station.nunique()}")
display_row_values(df_station)

24

Station ID      24
Station Zone    24
Lat             24
Lon             24
dtype: int64
Station ID  |  Station Zone                              |  Lat    |  Lon    
--------------------------------------------------------------------------
41008       |  grays reef                                |  31.4   |  -80.87 
41044       |  ne st martin                              |  21.58  |  -58.63 
41049       |  south bermuda                             |  27.5   |  -62.27 
42001       |  mid gulf                                  |  25.93  |  -89.66 
42002       |  west gulf                                 |  25.95  |  -93.78 
42036       |  west tampa                                |  28.5   |  -84.5  
42056       |  yucatan basin                             |  19.82  |  -84.98 
42058       |  central caribbean                         |  14.51  |  -75.15 
44007       |  portland                                  |  43.52  |  -70.14 
44020       |  nantucket sound                      

In [79]:
try:
    create_table_in_mysql(df=df_station, engine=engine, table_name='dim_station')
    insert_new_rows(df=df_station, engine=engine, table='dim_station', ref_column='Station ID')
except Exception as e:
    print(e)

⚠️ La table 'dim_station' existe déjà dans MySQL.
🔍 Connexion à la base de données en cours...
🧮 Vérification si la table est vide...
⚡ La table est vide. Insertion de toutes les données avec suivi...
⏳ Insertion en cours : 1/24

⏳ Insertion en cours : 24/24
✅ Toutes les lignes ont été insérées dans la table.


In [80]:
print(f"{df_time.shape[0]}\n\n{df_time.nunique()}")
display_row_values(df_time)

1093

Date ID      1093
Year            1
Month           3
Day            31
Hour           24
DayOfWeek       7
DayPeriod       4
dtype: int64
Date ID     |  Year  |  Month  |  Day  |  Hour  |  DayOfWeek  |  DayPeriod
--------------------------------------------------------------------
2025041112  |  2025  |  4      |  11   |  12    |  4          |  Afternoon
2025041111  |  2025  |  4      |  11   |  11    |  4          |  Morning  
2025041110  |  2025  |  4      |  11   |  10    |  4          |  Morning  
2025041109  |  2025  |  4      |  11   |  9     |  4          |  Morning  
2025041108  |  2025  |  4      |  11   |  8     |  4          |  Morning  
2025041107  |  2025  |  4      |  11   |  7     |  4          |  Morning  
2025041106  |  2025  |  4      |  11   |  6     |  4          |  Morning  
2025041105  |  2025  |  4      |  11   |  5     |  4          |  Night    
2025041104  |  2025  |  4      |  11   |  4     |  4          |  Night    
2025041103  |  2025  |  4      |  11

In [81]:
def insert_new_rows(df: pd.DataFrame, engine, table: str, ref_column: str):
    """
    Insère les nouvelles lignes dans la table MySQL après avoir vérifié si les IDs uniques existent déjà.

    Args:
    - df (pd.DataFrame): Le DataFrame contenant les nouvelles données.
    - engine (SQLAlchemy Engine): L'engine SQLAlchemy pour se connecter à la base de données.
    - table (str): Le nom de la table dans laquelle insérer les données.
    - ref_column (str): Le nom de la colonne à utiliser comme référence (ID unique).
    """
    try:
        print("🔍 Connexion à la base de données en cours...")

        with engine.connect() as connection:
            # Vérifier si la table est vide
            print("🧮 Vérification si la table est vide...")
            check_empty_sql = f"SELECT COUNT(*) FROM `{table}`"
            result = connection.execute(text(check_empty_sql))
            row_count = result.fetchone()[0]

            if row_count == 0:
                # La table est vide, insérer toutes les données avec compteur
                print("⚡ La table est vide. Insertion de toutes les données avec suivi...")
                total = len(df)
                inserted = 0
                for _, row in df.iterrows():
                    row_df = pd.DataFrame([row])
                    row_df.to_sql(table, con=engine, if_exists='append', index=False)
                    inserted += 1
                    sys.stdout.write(f"\r⏳ Insertion en cours : {inserted}/{total}")
                    sys.stdout.flush()
                print("\n✅ Toutes les lignes ont été insérées dans la table.")
            else:
                # Sinon, récupérer les IDs existants dans la table
                print(f"🔎 Récupération des {ref_column} existants dans la table...")
                check_existing_ids_sql = f"SELECT `{ref_column}` FROM `{table}`"
                result = connection.execute(text(check_existing_ids_sql))
                existing_ids = {row[0] for row in result.fetchall()}
                print(f"🧑‍💻 {len(existing_ids)} {ref_column} existants ont été trouvés dans la table.")

                # Filtrage des lignes du DataFrame
                print("🔄 Filtrage des nouvelles lignes à insérer...")
                new_rows_df = df[~df[ref_column].isin(existing_ids)]

                if not new_rows_df.empty:
                    total = len(new_rows_df)
                    print(f"🚀 Insertion de {total} nouvelles lignes...")
                    inserted = 0
                    for _, row in new_rows_df.iterrows():
                        row_df = pd.DataFrame([row])
                        row_df.to_sql(table, con=engine, if_exists='append', index=False)
                        inserted += 1
                        sys.stdout.write(f"\r⏳ Insertion en cours : {inserted}/{total}")
                        sys.stdout.flush()
                    print("\n✅ Insertion terminée avec succès.")
                else:
                    print(f"⚠️ Aucune nouvelle ligne à insérer, tous les {ref_column} sont déjà présents.")
    except Exception as e:
        print(f"❌ Erreur lors de l'insertion dans la table '{table}': {e}")


In [84]:
try:
    create_table_in_mysql(df=df_time, engine=engine, table_name='dim_time')
    insert_new_rows(df=df_time, engine=engine, table='dim_time', ref_column='Date ID')
except Exception as e:
    print(e)

⚠️ La table 'dim_time' existe déjà dans MySQL.
🔍 Connexion à la base de données en cours...
🧮 Vérification si la table est vide...
🔎 Récupération des Date ID existants dans la table...
🧑‍💻 1093 Date ID existants ont été trouvés dans la table.
🔄 Filtrage des nouvelles lignes à insérer...
⚠️ Aucune nouvelle ligne à insérer, tous les Date ID sont déjà présents.


In [85]:
print(f"{df_facts_meteo.shape[0]}\n\n{df_facts_meteo.nunique()}")
display_row_values(df_facts_meteo)

25419

ID Unique                   25419
T°(C°)                       1810
Relative Humidity (%)          72
Dew Point (°C)                415
Precipitations (mm)            69
Sea Level Pressure (hPa)      582
Low Clouds (%)                101
Middle Clouds (%)             101
High Clouds (%)               101
Visibility (km)              1264
Wind Speed (10m)             3648
Wind Direction (°)             36
Wind Gusts (km/h)              28
Barometer Elevation (m)         3
Air T° Height (m)               3
Date ID                      1093
Station ID                     24
dtype: int64
ID Unique        |  T°(C°)  |  Relative Humidity (%)  |  Dew Point (°C)  |  Precipitations (mm)  |  Sea Level Pressure (hPa)  |  Low Clouds (%)  |  Middle Clouds (%)  |  High Clouds (%)  |  Visibility (km)  |  Wind Speed (10m)  |  Wind Direction (°)  |  Wind Gusts (km/h)  |  Barometer Elevation (m)  |  Air T° Height (m)  |  Date ID     |  Station ID
--------------------------------------------------

In [86]:
try:
    create_table_in_mysql(df=df_facts_meteo, engine=engine, table_name='facts_meteo')
    insert_new_rows(df=df_facts_meteo, engine=engine, table='facts_meteo', ref_column='ID Unique')
except Exception as e:
    print(e)

✅ Table 'facts_meteo' créée avec succès dans MySQL.
🔍 Connexion à la base de données en cours...
🧮 Vérification si la table est vide...
⚡ La table est vide. Insertion de toutes les données avec suivi...
⏳ Insertion en cours : 25419/25419
✅ Toutes les lignes ont été insérées dans la table.


In [87]:
print(f"{df_facts_ocean.shape[0]}\n\n{df_facts_ocean.nunique()}")
display_row_values(df_facts_ocean)

25419

ID Unique                      25419
Wave Height (m)                   45
Average Wave Period (s)           65
Dominant Wave Direction (°)      149
Water T° (°C)                    278
Water Depth (m)                   24
Sea Temperature Depth (m)          3
Date ID                         1093
Station ID                        24
dtype: int64
ID Unique        |  Wave Height (m)  |  Average Wave Period (s)  |  Dominant Wave Direction (°)  |  Water T° (°C)  |  Water Depth (m)  |  Sea Temperature Depth (m)  |  Date ID     |  Station ID
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
410082025041112  |  0.9              |  4.8                      |  110.0                        |  19.2           |  16               |  2                          |  2025041112  |  41008     
410082025041111  |  0.9              |  4.8                      |  110.0

In [88]:
try:
    create_table_in_mysql(df=df_facts_ocean, engine=engine, table_name='facts_ocean')
    insert_new_rows(df=df_facts_ocean, engine=engine, table='facts_ocean', ref_column='ID Unique')
except Exception as e:
    print(e)

✅ Table 'facts_ocean' créée avec succès dans MySQL.
🔍 Connexion à la base de données en cours...
🧮 Vérification si la table est vide...
⚡ La table est vide. Insertion de toutes les données avec suivi...
⏳ Insertion en cours : 25419/25419
✅ Toutes les lignes ont été insérées dans la table.


In [ ]:
# # Configurer le logging pour capturer les erreurs et l'état des processus
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# # Début du processus d'insertion des données
# print("🚀 Début de l'insertion des données dans la base de données...\n")

# # Compteur pour les commits par lots
# batch_size = 100
# station_batch = []
# date_batch = []
# meteo_batch = []
# ocean_batch = []

# # Fonction pour insérer des données par lots dans une table
# def commit_batch(batch, table):
#     if batch:
#         try:
#             session.execute(table.insert(), batch)
#             session.commit()
#             logging.info(f"✅ {len(batch)} lignes insérées avec succès dans {table.name}.")
#         except Exception as e:
#             logging.error(f"❌ Erreur lors de l'insertion dans {table.name}: {e}")
#             session.rollback()  # Ajouter un rollback en cas d'erreur
#         finally:
#             batch.clear()  # Réinitialiser le batch après commit

# ####################### 1. Insertion unique dans la table dimstation ##################################################

# print("🔄 Insertion des données dans la table 'dimstation' en cours...")
# for idx, row in df_final.iterrows():
#     try:
#         # Vérifier si la station existe déjà
#         existing_station = session.query(dimstation).filter_by(Station_ID=row['Station ID']).first()
#         if not existing_station:
#             station_batch.append({
#                 'Station_ID': row['Station ID'],  # Ajouter le Station_ID
#                 'Station_Zone': row['Station Zone'],
#                 'Lat': row['Lat'],
#                 'Lon': row['Lon'],
#             })

#         # Commit après chaque batch de stations
#         if len(station_batch) >= batch_size:
#             commit_batch(station_batch, dimstation)

#     except Exception as e:
#         logging.error(f"❌ Erreur lors du traitement de la station {row['Station ID']}: {e}")
#         session.rollback()  # Faire un rollback en cas d'erreur

# # Commit final pour la dernière batch de stations
# commit_batch(station_batch, dimstation)
# print("\n")

# ######################### 2. Insertion unique dans la table dimdate ################################################

# print("🔄 Insertion des données dans la table 'dimdate' en cours...")
# for idx, row in df_final.iterrows():
#     try:
#         # Récupérer les valeurs pour créer une combinaison unique
#         year = row['Datetime'].year
#         month = row['Datetime'].month
#         day = row['Datetime'].day
#         hour = row['Datetime'].hour  # Récupérer l'heure
#         day_of_week = row['Datetime'].weekday()  # Jour de la semaine (0 = Lundi, 6 = Dimanche)
        
#         # Vérifier si une combinaison Year, Month, Day, DayOfWeek, Hour existe déjà
#         existing_date = session.query(dimdate).filter_by(
#             Year=year, Month=month, Day=day, DayOfWeek=day_of_week, Hour=hour).first()
        
#         if not existing_date:
#             date_batch.append({
#                 'Year': year,
#                 'Month': month,
#                 'Day': day,
#                 'Hour': hour,  # Ajouter l'heure à la combinaison unique
#                 'DayOfWeek': day_of_week,
#                 'DayPeriod': row['Day Period'],
#             })

#         # Commit après chaque batch de dates
#         if len(date_batch) >= batch_size:
#             commit_batch(date_batch, dimdate)

#     except Exception as e:
#         logging.error(f"❌ Erreur lors du traitement de la date pour la station {row['Station ID']}: {e}")
#         session.rollback()  # Faire un rollback en cas d'erreur

# # Commit final pour la dernière batch de dates
# commit_batch(date_batch, dimdate)
# print("\n")

# ################### 3. Insertion dans la table factmeteorologicaldata ##################################################

# print("🔄 Insertion des données dans la table 'factmeteorologicaldata' en cours...")
# for idx, row in df_final.iterrows():
#     try:
#         # Vérifier si la ligne météorologique existe déjà (en fonction de Station_ID et Date_ID)
#         existing_meteo = session.query(factmeteorologicaldata).filter_by(
#             Station_ID=row['Station ID'], Date_ID=row['Datetime'].strftime('%Y%m%d%H')).first()  # Ajouter l'heure à la Date_ID
#         if not existing_meteo:
#             meteo_batch.append({
#                 'Datetime': row['Datetime'],
#                 'T_C': row['T°(C°)'],
#                 'Relative_Humidity': row['Relative Humidity (%)'],
#                 'Dew_Point': row['Dew Point (°C)'],
#                 'Precipitations': row['Precipitations (mm)'],
#                 'Sea_Level_Pressure': row['Sea Level Pressure (hPa)'],
#                 'Low_Clouds': row['Low Clouds (%)'],
#                 'Middle_Clouds': row['Middle Clouds (%)'],
#                 'High_Clouds': row['High Clouds (%)'],
#                 'Visibility': row['Visibility (km)'],
#                 'Wind_Speed': row['Wind Speed (10m)'],
#                 'Wind_Direction': row['Wind Direction (°)'],
#                 'Wind_Gusts': row['Wind Gusts (km/h)'],
#                 'Barometer_Elevation': row['Barometer Elevation'],
#                 'Air_T_Height': row['Air T° Height'],
#                 'Station_ID': row['Station ID'],
#                 'Date_ID': row['Datetime'].strftime('%Y%m%d%H'),  # Ajouter l'heure dans Date_ID
#             })

#         # Commit après chaque batch de données météorologiques
#         if len(meteo_batch) >= batch_size:
#             commit_batch(meteo_batch, factmeteorologicaldata)

#     except Exception as e:
#         logging.error(f"❌ Erreur lors du traitement des données météorologiques pour la station {row['Station ID']}: {e}")
#         session.rollback()  # Faire un rollback en cas d'erreur

# # Commit final pour la dernière batch de données météorologiques
# commit_batch(meteo_batch, factmeteorologicaldata)
# print("\n")

# ###################### 4. Insertion dans la table factoceanographicdata ############################################

# print("🔄 Insertion des données dans la table 'factoceanographicdata' en cours...")
# for idx, row in df_final.iterrows():
#     try:
#         # Vérifier si la ligne océanographique existe déjà (en fonction de Station_ID et Date_ID)
#         existing_ocean = session.query(factoceanographicdata).filter_by(
#             Station_ID=row['Station ID'], Date_ID=row['Datetime'].strftime('%Y%m%d%H')).first()  # Ajouter l'heure à la Date_ID
#         if not existing_ocean:
#             ocean_batch.append({
#                 'Datetime': row['Datetime'],
#                 'Wave_Height': row['Wave Height (m)'],
#                 'Average_Wave_Period': row['Average Wave Period (s)'],
#                 'Dominant_Wave_Direction': row['Dominant Wave Direction (°)'],
#                 'Water_Temp': row['Water T°'],
#                 'Water_Depth': row['Water Depth (m)'],
#                 'Sea_Temp_Depth': row['Sea Temperature Depth (m)'],
#                 'Station_ID': row['Station ID'],
#                 'Date_ID': row['Datetime'].strftime('%Y%m%d%H'),  # Ajouter l'heure dans Date_ID
#             })

#         # Commit après chaque batch de données océanographiques
#         if len(ocean_batch) >= batch_size:
#             commit_batch(ocean_batch, factoceanographicdata)

#     except Exception as e:
#         logging.error(f"❌ Erreur lors du traitement des données océanographiques pour la station {row['Station ID']}: {e}")
#         session.rollback()  # Faire un rollback en cas d'erreur

# # Commit final pour la dernière batch de données océanographiques
# commit_batch(ocean_batch, factoceanographicdata)
# print("\n")

# # Fermer la session pour libérer les ressources
# session.close()

# # Message final
# print("🎉 Tous les processus d'insertion sont terminés avec succès ! Les données ont été insérées dans la base de données.")